# Data Cleaing

This notebook is reads the `car_prices.csv` dataset and produces a cleaned dataset, eliminating any outliers, abnormal values and missing values.


In [1]:
import pandas as pd
import numpy as np
import json
import csv
import requests
import time

In [2]:
data = pd.read_csv("car_prices.csv")

## Missing Values


In [3]:
print(
    f"Number of rows with missing values: {data.isna().any(axis=1).value_counts()[True]} = {round(data.isna().any(axis=1).value_counts()[True] / data.shape[0] * 100 , 2)}%"
)
print("Missing Values in individula columns ")
print(data.isna().sum())

Number of rows with missing values: 86512 = 15.48%
Missing Values in individula columns 
year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
vin                 4
state               0
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64


# Recovering Missing values

There are a lot of missing values, but since this dataset has the VIN(Vehicle Identification Number) for all except 4 rows, I can look the number to find and fill in any missing details. Speaking of the 4 rows, I can check them and fill them in with dummy VINs.


In [4]:
data[data["vin"].isna()]

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
461612,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,NaN,3vwd17aj3fm259017,NaN,46.0,2711,white,black,NaN,14250.0,14000
505299,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj7fm222388,NaN,36.0,20379,silver,black,NaN,13600.0,13500
529009,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj8fm298895,NaN,2.0,2817,red,black,NaN,13750.0,12200
551222,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,NaN,3vwd17aj8fm239622,NaN,2.0,9562,silver,black,NaN,13200.0,12100


So the missing VINs were placed in the state column so i jst need to correct them I guess...


In [5]:
rows = data[data["vin"].isna()].index

for row in rows:
    data.at[row, "vin"] = data.at[row, "state"]
    data.at[row, "state"] = np.nan

data.loc[rows]

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
461612,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,Sedan,3vwd17aj3fm259017,NaN,NaN,46.0,2711,white,black,NaN,14250.0,14000
505299,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,3vwd17aj7fm222388,NaN,NaN,36.0,20379,silver,black,NaN,13600.0,13500
529009,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,3vwd17aj8fm298895,NaN,NaN,2.0,2817,red,black,NaN,13750.0,12200
551222,2015,Volkswagen,Jetta,SE PZEV w/Connectivity,Navitgation,sedan,3vwd17aj8fm239622,NaN,NaN,2.0,9562,silver,black,NaN,13200.0,12100


In [6]:
data.isna().sum()

year                0
make            10301
model           10399
trim            10651
body            13195
transmission    65352
vin                 0
state               4
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64

now that I have all the VINs, I can reach out this [API](https://vpic.nhtsa.dot.gov/api/Home) to fetch the missing details. Feel free to click [here](https://vpic.nhtsa.dot.gov/api/vehicles/decodevin/5UXWX7C5*BA?format=json&modelyear=2011) to check a sample response. Spoiler alert, its huge and mostly unreadable. Hence, I used ChatGPT to generate a snippet to fetch all the missing data and store it into a csv file.


The `raise` statement on the top is because I don't want to put 10K request to the Free api Each time I hit Run all.


In [7]:
# Uncomment the following lines in this cell to re-build the dataset with missing details.

# vin_series = data["vin"].where(data["make"].isna()).dropna().tolist()

# csv_file = "vehicle_info.csv"

# with open(csv_file, mode="w", newline="") as file:

#     fieldnames = ["vin", "make", "model", "trim", "transmission", "color", "interior"]
#     writer = csv.DictWriter(file, fieldnames=fieldnames)

#     writer.writeheader()

#     total_vins = len(vin_series)

#     for index, vin in enumerate(vin_series):

#         url = f"https://vpic.nhtsa.dot.gov/api/vehicles/decodevin/{vin}?format=json"

#         try:

#             response = requests.get(url)

#             if response.status_code == 200:

#                 details = response.json()

#                 vehicle_info = {
#                     "VIN": vin,
#                     "Make": None,
#                     "Model": None,
#                     "Trim": None,
#                     "Transmission": None,
#                     "Color": None,
#                     "Interior": None,
#                 }

#                 for item in details["Results"]:
#                     if item["Variable"] == "Make":
#                         vehicle_info["Make"] = item["Value"]
#                     elif item["Variable"] == "Model":
#                         vehicle_info["Model"] = item["Value"]
#                     elif item["Variable"] == "Trim":
#                         vehicle_info["Trim"] = item["Value"]
#                     elif item["Variable"] == "Transmission Style":
#                         vehicle_info["Transmission"] = item["Value"]
#                     elif item["Variable"] == "Color":
#                         vehicle_info["Color"] = item["Value"]
#                     elif item["Variable"] == "Interior":
#                         vehicle_info["Interior"] = item["Value"]

#                 writer.writerow(vehicle_info)

#             else:
#                 print(
#                     f"Failed to retrieve data for VIN {vin}. Status code: {response.status_code}"
#                 )

#         except Exception as e:
#             print(f"Fatal error processing VIN {vin}: {e}")

#         print(f"Processed {index + 1} of {total_vins} VINs.")

#         time.sleep(0.1)

# print(f"Data has been written to {csv_file}")

Fill in the missing data from the extracted details.


In [8]:
tobeFilled = data[data["make"].isna()]
filler = pd.read_csv("vehicle_info.csv")
filled = pd.merge(tobeFilled, filler, on="vin", how="inner")
filled.head(3)

,year,make_x,model_x,trim_x,body,transmission_x,vin,state,condition,odometer,...,seller,mmr,sellingprice,saledate,make_y,model_y,trim_y,transmission_y,color_y,interior_y
0,2012,NaN,NaN,NaN,NaN,automatic,wbakb8c51cc964387,ca,38.0,23208.0,...,financial services remarketing (lease),47200.0,46000.0,Thu Feb 26 2015 04:30:00 GMT-0800 (PST),BMW,750Li,LWB,NaN,NaN,NaN
1,2012,NaN,NaN,NaN,NaN,automatic,wbakb8c53cc964410,ca,33.0,19785.0,...,financial services remarketing (lease),49500.0,46000.0,Thu Feb 12 2015 04:30:00 GMT-0800 (PST),BMW,750Li,LWB,NaN,NaN,NaN
2,2012,NaN,NaN,NaN,NaN,automatic,wbakb8c54cc964089,ca,37.0,48424.0,...,financial services remarketing (lease),42300.0,43000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST),BMW,750Li,LWB,NaN,NaN,NaN


In [9]:
filled["transmission"] = filled["transmission_x"].combine_first(
    filled["transmission_y"]
)
filled["make"] = filled["make_x"].combine_first(filled["make_y"])
filled["model"] = filled["model_x"].combine_first(filled["model_y"])
filled["trim"] = filled["trim_x"].combine_first(filled["trim_y"])

final = filled[["vin", "transmission", "make", "model", "trim"]]

final = final.drop_duplicates(subset="vin", keep="first")

In [10]:
final.isna().sum()

vin                0
transmission    1335
make               0
model             54
trim            3743
dtype: int64

In [11]:
data.set_index("vin", inplace=True)
final.set_index("vin", inplace=True)
data.update(final)
data.reset_index(inplace=True)

In [12]:
data.isna().sum()

vin                 0
year                0
make             1555
model            1707
trim             5707
body            13195
transmission    65207
state               4
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64

There are some missing values in the dataset which is because that detail extractor cell stopped somewhere near the end, I would have to rerun it to fetch the rest of the details. Now I would focus on Transmission.


In [13]:
data["transmission"].value_counts(dropna=False)

transmission
automatic    475926
NaN           65207
manual        17544
Automatic       119
Manual           15
sedan            15
Sedan            11
Name: count, dtype: int64

In [14]:
data["transmission"] = data["transmission"].replace({"Sedan": np.nan})
data["transmission"] = data["transmission"].replace({"sedan": np.nan})
data["transmission"] = data["transmission"].replace({"automatic": "Automatic"})
data["transmission"] = data["transmission"].replace({"manual": "Manual"})
data["transmission"] = data["transmission"].replace({np.nan: "UnSpecified"})

In [15]:
data["transmission"].value_counts(dropna=False)

transmission
Automatic      476045
UnSpecified     65233
Manual          17559
Name: count, dtype: int64

In [16]:
data.isna().sum()

vin                 0
year                0
make             1555
model            1707
trim             5707
body            13195
transmission        0
state               4
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                38
sellingprice       12
saledate           12
dtype: int64

Going For Selling price and date first, because it has few missing values.


In [17]:
data[data["sellingprice"].isna()]

,vin,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
306447,5npeb4ac6dh687932,2013,Hyundai,Sonata,GLS,Sedan,Automatic,il,19.0,37254.0,gray,gray,kfl llc,NaN,NaN,NaN
406524,2c3ccaag9dh723146,2013,Chrysler,300,Base,Sedan,Automatic,il,27.0,44208.0,silver,black,kfl llc,NaN,NaN,NaN
422358,5npeb4ac3dh735368,2013,Hyundai,Sonata,GLS,Sedan,UnSpecified,az,28.0,44299.0,blue,gray,kfl llc,NaN,NaN,NaN
429644,1c3ccbbb9dn718531,2013,Chrysler,200,Touring,Sedan,Automatic,il,25.0,47344.0,white,black,kfl llc,NaN,NaN,NaN
429660,1c3cdzcg5dn741076,2013,Dodge,Avenger,SXT,Sedan,Automatic,il,34.0,44525.0,gray,black,kfl llc,NaN,NaN,NaN
429786,kmhct4ae7du450468,2013,Hyundai,Accent,GLS,Sedan,Automatic,il,3.0,43135.0,silver,gray,kfl llc,NaN,NaN,NaN
431905,kndjt2a5xd7765280,2013,Kia,Soul,Base,Wagon,Automatic,va,19.0,29465.0,silver,black,kfl llc,NaN,NaN,NaN
457035,5npdh4ae3eh504144,2014,Hyundai,Elantra,SE,Sedan,Automatic,il,19.0,20775.0,white,—,kfl llc,NaN,NaN,NaN
457803,2c3cdxbg6dh688757,2013,Dodge,Charger,SE,Sedan,Automatic,il,19.0,45355.0,white,black,kfl llc,NaN,NaN,NaN
522495,3fahp0jg5cr241590,2012,Ford,Fusion,SEL,sedan,Automatic,va,26.0,51648.0,black,gray,kfl llc,NaN,NaN,NaN


It seems like all the data is present except the sale details. So I **Assume** that these vehicles are not sold yet and are ready to be sold.


In [18]:
data["mmr"] = data["mmr"].replace({np.nan: 0})
data["sellingprice"] = data["sellingprice"].replace({np.nan: 0})
data["saledate"] = data["saledate"].replace({np.nan: "TO BE SOLD"})

In [19]:
data.isna().sum()

vin                 0
year                0
make             1555
model            1707
trim             5707
body            13195
transmission        0
state               4
condition       11820
odometer           94
color             749
interior          749
seller              0
mmr                 0
sellingprice        0
saledate            0
dtype: int64

Again, I **Assume** that a `null` value in `odometer` means that the vehicle have 0 mileage.


In [20]:
data["odometer"] = data["odometer"].replace({np.nan: 0})

Now for the `condition`, I impute with the mean value. Since it is a very large dataset anyways, 11.8k or 2.11% of the rows imputed with mean should not matter that much...


In [21]:
data["condition"] = data["condition"].replace({np.nan: data["condition"].mean()})

In [22]:
data.isna().sum(), data.isna().any(axis=1).sum(), data.shape

(vin                 0
 year                0
 make             1555
 model            1707
 trim             5707
 body            13195
 transmission        0
 state               4
 condition           0
 odometer            0
 color             749
 interior          749
 seller              0
 mmr                 0
 sellingprice        0
 saledate            0
 dtype: int64,
 np.int64(14020),
 (558837, 16))

In [23]:
print(
    f"Number of rows with missing values: {data.isna().any(axis=1).value_counts()[True]} = {round(data.isna().any(axis=1).value_counts()[True] / data.shape[0] * 100 , 2)}%"
)

Number of rows with missing values: 14020 = 2.51%


Well this final bit of missing values account for just 2.5%, so I am gonna just brush it under the rug... So after a quick re-shuffle in the column order, the dataset should be ready.


In [24]:
data.dropna(inplace=True)

Now I also find that there are rows where the odometer is less than 100 but the condition is in despair.


In [25]:
unreliableConditionOdo = (
    data.where(data["odometer"] < 100).where(data["condition"] < 45).dropna()
)
unreliableConditionOdo

,vin,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
61,wauacgff6f1029938,2015.0,Audi,A3,1.8 TFSI Premium,Sedan,Automatic,ca,2.000000,31.0,white,black,enterprise vehicle exchange / tra / rental / t...,24200.0,17300.0,Tue Dec 23 2014 12:00:00 GMT-0800 (PST)
316,kmhec4a46da099621,2013.0,Hyundai,Sonata Hybrid,Limited,Sedan,Automatic,ca,5.000000,20.0,silver,gray,hyundai motor america/co car,20300.0,19900.0,Tue Dec 16 2014 12:00:00 GMT-0800 (PST)
338,5xyzu3la5dg079070,2013.0,Hyundai,Santa Fe,Sport 2.0T,SUV,Automatic,ca,5.000000,66.0,silver,black,hyundai motor america/co car,23100.0,20800.0,Tue Dec 16 2014 12:00:00 GMT-0800 (PST)
697,1n4aa5apxdc839917,2013.0,Nissan,Maxima,3.5 S,Sedan,Automatic,ca,1.000000,1.0,gray,black,enterprise vehicle exchange / tra / rental / t...,16950.0,1000.0,Tue Jan 13 2015 13:00:00 GMT-0800 (PST)
1000,1c3ccbab7cn236141,2012.0,Chrysler,200,LX,Sedan,Automatic,ca,30.672365,1.0,gray,black,santander consumer,10450.0,1300.0,Tue Dec 16 2014 13:00:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556056,1n4al3ap2fc206202,2015.0,Nissan,Altima,2.5 S,sedan,Automatic,tx,5.000000,19.0,gray,black,rlb investments,16100.0,16800.0,Wed Jun 17 2015 03:20:00 GMT-0700 (PDT)
557306,1b7hc16x31s209808,2001.0,Dodge,Ram Pickup 1500,ST,regular cab,Automatic,ms,1.000000,1.0,white,gray,select remarketing group llc/midwest title loans,1750.0,300.0,Thu Jun 18 2015 05:02:00 GMT-0700 (PDT)
557361,3czrm3h33eg717730,2014.0,Honda,CR-V,LX,suv,Automatic,nv,42.000000,14.0,silver,gray,american honda motor company,19750.0,19000.0,Fri Jun 19 2015 05:00:00 GMT-0700 (PDT)
558359,3gnca53v39s567637,2009.0,Chevrolet,HHR,LT,wagon,Automatic,pa,34.000000,0.0,blue,gray,morgan automotive inc,5700.0,9700.0,Fri Jun 19 2015 02:15:00 GMT-0700 (PDT)


Since its just 1.6k rows, I would simply get rid of them


In [26]:
data.drop(index=unreliableConditionOdo.index, inplace=True)

I would also double the condition to make it easier to interpret(out of 100 rather than 50)


In [27]:
data["condition"] = data["condition"] * 2

In [28]:
data.head()

,vin,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,5xyktca69fg566472,2015,Kia,Sorento,LX,SUV,Automatic,ca,10.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,5xyktca69fg561319,2015,Kia,Sorento,LX,SUV,Automatic,ca,10.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,wba3c1c51ek116351,2014,BMW,3 Series,328i SULEV,Sedan,Automatic,ca,90.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,yv1612tb4f1310987,2015,Volvo,S60,T5,Sedan,Automatic,ca,82.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,wba6b2c57ed129731,2014,BMW,6 Series Gran Coupe,650i,Sedan,Automatic,ca,86.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


The missing values in the `color` and `interior` are marked as "-" so I replace them with "Unspecified"


In [29]:
data["color"].unique()

array(['white', 'gray', 'black', 'red', 'silver', 'blue', 'brown',
       'beige', 'purple', 'burgundy', '—', 'gold', 'yellow', 'green',
       'charcoal', 'orange', 'off-white', 'turquoise', 'pink', 'lime'],
      dtype=object)

In [30]:
data["color"] = data["color"].replace({"—": "UnSpecified"})

In [31]:
data["interior"].unique()

array(['black', 'beige', 'tan', '—', 'gray', 'brown', 'burgundy', 'white',
       'silver', 'off-white', 'red', 'yellow', 'blue', 'green', 'purple',
       'orange', 'gold'], dtype=object)

In [32]:
data["interior"] = data["interior"].replace({"—": "UnSpecified"})

Now the state column has only initials. The next step replaces them with full forms, for better readability.


In [33]:
data["state"].unique()

array(['ca', 'tx', 'pa', 'mn', 'az', 'wi', 'tn', 'md', 'ne', 'nv', 'oh',
       'fl', 'mi', 'nj', 'ga', 'va', 'sc', 'nc', 'in', 'il', 'co', 'mo',
       'ut', 'ny', 'ma', 'pr', 'or', 'la', 'wa', 'hi', 'qc', 'ab', 'on',
       'ok', 'ms', 'al', 'nm', 'ns'], dtype=object)

In [34]:
states = {
    "al": "Alabama",
    "ak": "Alaska",
    "az": "Arizona",
    "ar": "Arkansas",
    "ca": "California",
    "co": "Colorado",
    "ct": "Connecticut",
    "de": "Delaware",
    "fl": "Florida",
    "ga": "Georgia",
    "hi": "Hawaii",
    "id": "Idaho",
    "il": "Illinois",
    "in": "Indiana",
    "ia": "Iowa",
    "ks": "Kansas",
    "ky": "Kentucky",
    "la": "Louisiana",
    "me": "Maine",
    "md": "Maryland",
    "ma": "Massachusetts",
    "mi": "Michigan",
    "mn": "Minnesota",
    "ms": "Mississippi",
    "mo": "Missouri",
    "mt": "Montana",
    "ne": "Nebraska",
    "nv": "Nevada",
    "nh": "New Hampshire",
    "nj": "New Jersey",
    "nm": "New Mexico",
    "ny": "New York",
    "nc": "North Carolina",
    "nd": "North Dakota",
    "oh": "Ohio",
    "ok": "Oklahoma",
    "or": "Oregon",
    "pa": "Pennsylvania",
    "pr": "Puerto Rico",
    "ri": "Rhode Island",
    "sc": "South Carolina",
    "sd": "South Dakota",
    "tn": "Tennessee",
    "tx": "Texas",
    "ut": "Utah",
    "vt": "Vermont",
    "va": "Virginia",
    "wa": "Washington",
    "wv": "West Virginia",
    "wi": "Wisconsin",
    "wy": "Wyoming",
    "qc": "Quebec",
    "ab": "Alberta",
    "on": "Ontario",
    "ns": "Nova Scotia",
}

data["state"] = data["state"].map(states)

In [35]:
data.columns

Index(['vin', 'year', 'make', 'model', 'trim', 'body', 'transmission', 'state',
       'condition', 'odometer', 'color', 'interior', 'seller', 'mmr',
       'sellingprice', 'saledate'],
      dtype='object')

Rearranging the columns to make a little more sense.


In [36]:
data = data[
    [
        "vin",
        "year",
        "make",
        "model",
        "trim",
        "body",
        "transmission",
        "odometer",
        "condition",
        "color",
        "interior",
        "state",
        "seller",
        "mmr",
        "sellingprice",
        "saledate",
    ]
]

Finally export the cleaned dataset.


In [37]:
data.to_csv("cleanedData.csv", index=False)